<a href="https://colab.research.google.com/github/Rula-Islait/Projects/blob/main/Copy_of_plant_village_attention_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds

In [ ]:
dataset, info = tfds.load("plant_village", as_supervised=True, with_info=True, split=["train"])
num_classes = info.features["label"].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/plant_village/incomplete.8WDVIQ_1.0.2/plant_village-train.tfrecord*...:   …

Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.2. Subsequent calls will reuse this data.


In [ ]:
IMG_SIZE = 128

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0
    label = tf.one_hot(label, num_classes)
    return image, label


In [ ]:
train_dataset = dataset[0].map(preprocess).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
class SequentialAttention(layers.Layer):
    def __init__(self, filters):
        super(SequentialAttention, self).__init__()
        self.conv = layers.Conv2D(filters, (1, 1), activation='sigmoid')

    def call(self, inputs):
        attention_map = self.conv(inputs)
        return inputs * attention_map


In [ ]:
class ReverseAttention(layers.Layer):
    def __init__(self, filters):
        super(ReverseAttention, self).__init__()
        self.conv = layers.Conv2D(filters, (1, 1), activation='sigmoid')

    def call(self, inputs):
        attention_map = self.conv(inputs)
        return inputs - (inputs * attention_map)

In [ ]:
class DynamicAttention(layers.Layer):
    def __init__(self, filters):
        super(DynamicAttention, self).__init__()
        self.fc = layers.Dense(filters, activation='sigmoid')

    def call(self, inputs):
        x = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)
        attention = self.fc(x)
        return inputs * attention


In [ ]:
from tensorflow.keras import layers, models

def build_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = ReverseAttention(128)(x)

    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = DynamicAttention(256)(x)

    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = SequentialAttention(512)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model


In [ ]:
input_shape = (IMG_SIZE, IMG_SIZE, 3)
model = build_model(input_shape, num_classes)

In [ ]:
optimizer = Adam(learning_rate=0.0003)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [ ]:
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=train_dataset,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
test_loss, test_acc = model.evaluate(train_dataset, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
print(f"Test loss: {test_loss}")

In [ ]:
model.save("plant_village_attention_model.h5")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model

def get_attention_model(full_model):
    attention_layers = []
    for layer in full_model.layers:
        if isinstance(layer, (ReverseAttention, DynamicAttention, SequentialAttention, DenseAttention)):
            attention_layers.append(layer.output)
    attention_model = Model(inputs=full_model.input, outputs=attention_layers)
    return attention_model

def visualize_attention_maps(attention_model, sample_image):
    sample_image_expanded = np.expand_dims(sample_image, axis=0)
    attention_outputs = attention_model.predict(sample_image_expanded)
    plt.figure(figsize=(15, 5))
    plt.subplot(1, len(attention_outputs) + 1, 1)
    plt.imshow(sample_image.squeeze(), cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    for idx, att_map in enumerate(attention_outputs):
        if len(att_map.shape) == 4:
            att_map = np.mean(att_map[0], axis=-1)
        att_map -= att_map.min()
        att_map /= att_map.max()
        plt.subplot(1, len(attention_outputs) + 1, idx + 2)
        plt.imshow(att_map, cmap='jet')
        plt.title(f'Attention Map {idx+1}')
        plt.axis('off')

    plt.show()

attention_model = get_attention_model(model)
random_idx = np.random.randint(0, len(x_test))
test_image = x_test[random_idx]
visualize_attention_maps(attention_model, test_image)
viualize